In [3]:
from simulations.drosselschwab import simulate_drosselschwab
import results
import data

# Then when saving a file you can use the path function like so:
results_path = results.path('p_param_2.csv')
data_path = data.path('p_param_2.csv')
print(results_path, data_path)

ModuleNotFoundError: No module named 'simulations'

In [2]:
L = 256
p = 0.01
f = 0.0001
steps = 10000

print("Start experiment:")

fire_sizes, grid = simulate_drosselschwab(L=L, p=p, f=f, steps=steps)

Start experiment:


NameError: name 'simulate_drosselschwab' is not defined

In [ ]:
import os
print(os.getcwd())